In [1]:

import sys
sys.path.insert(1, '../py')
from choropleth_utils import ChoroplethUtilities
from stats_scraping_utils import StatsScrapingUtilities
from storage import Storage
import numpy as np
import os
import re

s = Storage()
ssu = StatsScrapingUtilities(s=s)

In [6]:

world_map_df.columns.tolist()

['svg_version',
 'svg_width',
 'svg_height',
 'svg_id',
 'sodipodi_docname',
 'inkscape_inkscape_version',
 'svg_pagecolor',
 'svg_bordercolor',
 'svg_borderopacity',
 'inkscape_pageshadow',
 'inkscape_pageopacity',
 'inkscape_pagecheckerboard',
 'svg_showgrid',
 'inkscape_zoom',
 'inkscape_cx',
 'inkscape_cy',
 'inkscape_window_width',
 'inkscape_window_height',
 'inkscape_window_x',
 'inkscape_window_y',
 'inkscape_window_maximized',
 'inkscape_current_layer',
 'rdf_about',
 'svg_type',
 'svg_class',
 'outline_d',
 'circle_radius',
 'circle_y',
 'circle_x',
 'x',
 'y',
 'style',
 '{http://www.inkscape.org/namespaces/inkscape}label',
 'dominant-baseline',
 'text-anchor',
 '{http://sodipodi.sourceforge.net/DTD/sodipodi-0.dtd}role',
 'text_x',
 'text_y',
 'centroid_id',
 'centroid_x',
 'centroid_y',
 'country_color',
 'country_name',
 'label_line_d',
 'font_size',
 'dy']

In [20]:

world_map_df = s.load_object('world_map_df')
# df = world_map_df.set_index('country_code', drop=True)
us_stats_df = s.load_object('us_stats_df')
pwrindx_score_dict = s.load_object('firepower_df').set_index('country_name').pwrindx_score.to_dict()
world_map_df['log_pwrindx_score'] = world_map_df.country_name.map(lambda x: 3 + np.log(pwrindx_score_dict.get(x, 1)))
country_code_dict = s.load_object('all_countries_df').set_index('country_name').country_code.to_dict()
world_map_df['country_code'] = world_map_df.country_name.map(lambda x: country_code_dict.get(x))
wikipedia_title_dict = s.load_object('all_countries_df').set_index('country_name').wikipedia_title.to_dict()
world_map_df['wikipedia_title'] = world_map_df.country_name.map(lambda x: wikipedia_title_dict.get(x))
mask_series = world_map_df.country_name.isnull()
world_map_df = world_map_df[~mask_series]
numeric_column_name = 'log_pwrindx_score'
string_column_name = 'wikipedia_title'
one_country_df = world_map_df.set_index('country_name', drop=True).rename(columns={'country_code': 'district_abbreviation'})
c = ChoroplethUtilities(iso_3166_2_code='us', one_country_df=us_stats_df, all_countries_df=world_map_df)
svg_file_path = c.create_country_colored_labeled_map(numeric_column_name=numeric_column_name,
                                                     string_column_name=string_column_name,
                                                     one_country_df=one_country_df, cmap='viridis_r')
print(os.path.abspath(svg_file_path))

C:\Users\daveb\OneDrive\Documents\GitHub\StatsByCountry\saves\svg\US_log_pwrindx_score_wikipedia_title.svg


In [19]:

one_country_df.log_pwrindx_score

country_name
Sudan           3.291325
South Sudan     3.919167
Georgia         3.693847
Peru            2.706032
Burkina Faso    4.552148
                  ...   
Holy See        3.000000
Isle of Man     3.000000
Guam            3.000000
Singapore       2.530476
Tokelau         3.000000
Name: log_pwrindx_score, Length: 246, dtype: float64